In [31]:
from bs4 import BeautifulSoup
import pandas as pd
import os

In [32]:
# Takes in year and returns the last 2 digits that represent its year
# Examples:
# 2013 -> '13'
# 1999 -> '99'
# 2001 -> '01'
def get_last_year_digits(year):
    return str(year)[-2:]

In [33]:
# Create their respective folders if they're not there
if (not os.path.isdir('html-table')):
    os.mkdir('html-table')
if (not os.path.isdir('csv')):
    os.mkdir('csv')

In [34]:
# Extract HTML table of data from raw files
for year in range (1998, 2022 + 1):
    season_year_end = get_last_year_digits(year)
    season_year_start = get_last_year_digits(year - 1)
    
    team_stats_file = open(f'html-raw/{season_year_start}_{season_year_end}.html')
    team_stats_html = BeautifulSoup(team_stats_file)
    team_stats_table = team_stats_html.body.table
    team_stats_table_file = open(f'html-table/{season_year_start}_{season_year_end}.html', 'w')
    team_stats_table_file.write(str(team_stats_table))
    team_stats_table_file.close()

In [38]:
# Put the HTML table data into csv files
for year in range(1998, 2022 + 1):
    season_year_end = get_last_year_digits(year)
    season_year_start = get_last_year_digits(year - 1)
    
    table_file = open(f'html-table/{season_year_start}_{season_year_end}.html')
    table_html = BeautifulSoup(table_file)
    headers = []
    data = []
    for th in table_html.find('tr').find_all('th')[1:15]:
        headers.append(th.text.strip())
    # print(headers)
    for tr in table_html.find_all('tr')[1:]:
        row = []
        for td in tr.find_all('td')[1:]:
            row.append(td.text.strip())
        # print(row)
        data.append(row)
    df = pd.DataFrame(data, columns = headers)
    df['MIN'] = df['MIN'].str.replace(',', '') # Removes the comma that would ordinarily be in these values, pandas automatically corrects the type from str to int
    df.sort_values(by = 'WIN%', ascending = False)
    df.to_csv(f'csv/{season_year_start}_{season_year_end}.csv')

In [39]:
table = pd.read_csv('csv/21_22.csv', index_col=0)
print(table['FTARate'].mean())

0.2484666666666667
